In [3]:
import pandas as pd
import ast
from textblob import TextBlob
import re
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from langdetect import detect
from langcodes import Language

In [4]:
""""
games = pd.read_json('./output_steam_games.json', lines=True)

rows_r = []
with open('./australian_user_reviews.json', 'r', encoding='MacRoman') as f:
    for line in f.readlines():
        rows_r.append(ast.literal_eval(line))

reviews = pd.DataFrame(rows_r)

rows_i = []
with open('./australian_users_items.json', 'r', encoding='MacRoman') as f2:
    for line2 in f2.readlines():
        rows_i.append(ast.literal_eval(line2))

items = pd.DataFrame(rows_i)
""" 

games = pd.read_csv('./games.csv', sep=',')
reviews = pd.read_csv('./reviews.csv', sep=',')
items = pd.read_csv('./items.csv', sep=',')

In [3]:
games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro


In [7]:
reviews.head(1)

,user_id,user_url,reviews,date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","[None, None, None, None, None, None, None, Non..."


In [5]:
items.head(1)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [6]:
bck_reviews = reviews.copy()

In [70]:
reviews = bck_reviews.copy()

## Sentiment_analysis

In [10]:
reviews['reviews'] = reviews['reviews'].apply(ast.literal_eval)

In [72]:
def extract_item_id(review_list):
    item_id_list = []

    for review in review_list:
        if isinstance(review, dict) and 'item_id' in review:
            item_id_list.append(review['item_id'])
        else:
            item_id_list.append(None)

    return item_id_list

reviews['item_id'] = reviews['reviews'].apply(extract_item_id)

In [73]:
def extract_review(review_list):
    review_2_list = []

    for review in review_list:
        if isinstance(review, dict) and 'review' in review:
            review_2_list.append(review['review'])
        else:
            review_2_list.append(None)

    return review_2_list

reviews['review_2'] = reviews['reviews'].apply(extract_review)

In [74]:
def detect_language(text):
    try:
        lang_code = detect(text)
        lang_name = Language.get(lang_code).display_name()
        return lang_name
    except:
        return 'unknown'

In [75]:
reviews['language'] = reviews['review_2'].apply(lambda lst: detect_language(' '.join(lst)))

In [76]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [77]:
def preprocess_text(text: str, lang: str):
    text = text.lower()
    lang = lang.lower()
    
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    tokens = nltk.word_tokenize(text)
    
    try:
        stop_words = set(stopwords.words(lang))
    except:
        stop_words = set()
    
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens

In [78]:
reviews['review_clean'] = reviews.apply(lambda row: [preprocess_text(text, row['language']) for text in row['review_2']], axis=1)

In [79]:
def sentiment_analysis(review_list):
    sentiment_list = []

    for inner_list in review_list:
        if len(inner_list) == 0:
            sentiment_list.append('1')
        else:
            total_sentiment = 0
            for text in inner_list:
                sentiment = TextBlob(text).sentiment.polarity

                if sentiment > 0:
                    total_sentiment += 1
                elif sentiment < -0.2:
                    total_sentiment -= 1

            if total_sentiment > 0:
                sentiment_list.append('2')
            elif total_sentiment < 0:
                sentiment_list.append('0')
            else:
                sentiment_list.append('1')

    return sentiment_list

In [80]:
reviews['sentiment_analysis'] = reviews['review_clean'].apply(sentiment_analysis)

In [84]:
reviews = reviews.drop(columns=['review_2', 'review_clean', 'language'])

KeyError: "['review_2', 'review_clean'] not found in axis"

In [85]:
reviews.head()

,user_id,user_url,reviews,item_id,language,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","[1250, 22200, 43110]",English,"[2, 2, 1]"
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","[251610, 227300, 239030]",English,"[2, 2, 0]"
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...","[248820, 370360, 237930, 263360, 107200, 224500]",English,"[2, 2, 2, 0, 2, 2]"
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...","[250320, 20920, 204100, 224600, 207610, 108710]",English,"[2, 2, 2, 2, 0, 2]"
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...","[211420, 211820, 730, 204300]",English,"[1, 2, 2, 2]"


In [86]:
reviews.to_csv('./API_csv/sentiment_analysis_column.csv')

## def user_data

In [87]:
items.head()

,user_id,items_count,steam_id,user_url,items,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47..."


In [88]:
items['items'] = items['items'].apply(ast.literal_eval)
items['item_id'] = items['items'].apply(extract_item_id)

ValueError: malformed node or string: [{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_forever': 6, 'playtime_2weeks': 0}, {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_forever': 7, 'playtime_2weeks': 0}, {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '60', 'item_name': 'Ricochet', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '70', 'item_name': 'Half-Life', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_forever': 4733, 'playtime_2weeks': 0}, {'item_id': '240', 'item_name': 'Counter-Strike: Source', 'playtime_forever':

In [89]:
def extract_recommend(review_list):
    recommend_list = []

    for recommend in review_list:
        if isinstance(recommend, dict) and 'recommend' in recommend:
            recommend_list.append(recommend['recommend'])
        else:
            recommend_list.append(None)

    return recommend_list

reviews['recommend'] = reviews['reviews'].apply(extract_recommend)

In [90]:
def count_true(lst):
    return lst.count(True)

reviews['pct_recommend'] = reviews['recommend'].apply(lambda lst: lst.count(True) / len(lst) if len(lst) > 0 else 0)

In [91]:
items.head()

,user_id,items_count,steam_id,user_url,items,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47..."


In [92]:
reviews.head()

,user_id,user_url,reviews,item_id,language,sentiment_analysis,recommend,pct_recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","[1250, 22200, 43110]",English,"[2, 2, 1]","[True, True, True]",1.0
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014...","[251610, 227300, 239030]",English,"[2, 2, 0]","[True, True, True]",1.0
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',...","[248820, 370360, 237930, 263360, 107200, 224500]",English,"[2, 2, 2, 0, 2, 2]","[True, True, True, True, True, True]",1.0
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2...","[250320, 20920, 204100, 224600, 207610, 108710]",English,"[2, 2, 2, 2, 0, 2]","[True, True, True, True, True, True]",1.0
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',...","[211420, 211820, 730, 204300]",English,"[1, 2, 2, 2]","[True, True, True, True]",1.0


In [93]:
def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return value

games['price'] = games['price'].apply(convert_to_float)

In [94]:
def userdata(user_id: str):
    target_user_id = str(user_id)
    total_price = 0

    for index, row in items.iterrows():
        if target_user_id in row['user_id']:
            for item_id in row['item_id']:
                game_row = games[games['id']==float(item_id)]
                
                if not game_row.empty:
                    price = game_row['price'].values[0]
                    if price is not None:
                        if isinstance(price, (int, float)):
                            total_price += float(price)

    pct_recommend = reviews.loc[reviews['user_id'] == target_user_id, 'pct_recommend'].iloc[0]
    items_count = items.loc[items['user_id'] == target_user_id, 'items_count'].iloc[0]

    result_dict = {
                    'user': target_user_id,
                    'amount_spent': round(total_price, 2),
                    'recommendation_pct': pct_recommend,
                    'items_count': items_count
    }

    return result_dict

In [143]:
userdata("js41637")

{'user': 'js41637',
 'amount_spent': 8553.07,
 'recommendation_pct': 1.0,
 'items_count': 888}

In [95]:
items.to_csv('./API_csv/userdata_items.csv')
reviews.to_csv('./API_csv/userdata_reviews.csv')
games.to_csv('./API_csv/userdata_games.csv')

## def countreviews

In [11]:
def extract_date(review_list):
    date_list = []

    for date in review_list:
        if isinstance(date, dict) and 'posted' in date:
            date_list.append(date['posted'])
        else:
            date_list.append(None)

    return date_list

reviews['date'] = reviews['reviews'].apply(extract_date)

In [12]:
def modify_date_format(date_string):
    match = re.search(r'(\w+ \d+, \d+)', date_string)
    if match:
        modified_date = match.group(1)
        return modified_date
    return date_string

reviews['date'] = reviews['date'].apply(lambda date_list: [modify_date_format(date) for date in date_list])

In [13]:
def convert_to_datetime(date_list):
    date_pattern = re.compile(r'\w+\s\d+,\s\d{4}')
    date_objects = []
    
    for date in date_list:
        try:
            date_objects.append(pd.to_datetime(date_pattern.search(date).group()).date())
        except AttributeError:
            date_objects.append(None) 
    
    return date_objects

reviews['date'] = reviews['date'].apply(convert_to_datetime)

In [14]:
reviews.head(1)

,user_id,user_url,reviews,date
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2...","[2011-11-05, 2011-07-15, 2011-04-21]"


In [15]:
date_reviews = reviews.explode('date')

In [22]:
date_reviews['date'][0].dtype

dtype('O')

In [17]:
def countreviews(start_date: str, end_date: str):
    start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
    end_date = datetime.strptime(end_date, "%Y-%m-%d").date()

    filtered_dates = date_reviews[(date_reviews['date'] >= start_date) & (date_reviews['date'] <= end_date)]
    unique_user_count = filtered_dates['user_id'].nunique()
    unique_pct_recommend = filtered_dates['pct_recommend'].mean()

    result_dict = {
                    'start_date': start_date,
                    'end_date': end_date,
                    'users_count': unique_user_count,
                    'recommend_pct': unique_pct_recommend
    }

    return result_dict

In [18]:
countreviews("2011-01-01", "2011-02-01")

KeyError: 'pct_recommend'

In [104]:
date_reviews.to_csv('./API_csv/countreviews_date_reviews.csv')

## def genre

In [105]:
def extract_playtime_forever(item_list):
    playtime_forever_list = []

    for item_dic in item_list:
        if isinstance(item_dic, dict) and 'playtime_forever' in item_dic:
            playtime_forever_list.append(item_dic['playtime_forever'])
        else:
            playtime_forever_list.append(None)

    return playtime_forever_list

items['playtime_forever'] = items['items'].apply(extract_playtime_forever)

In [106]:
items.head()

,user_id,items_count,steam_id,user_url,items,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ...","[0, 0, 0, 220, 0, 0, 0, 62, 0, 0, 0, 0, 0, 0, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3...","[923, 0, 0, 158, 1323, 0, 90, 0, 234, 113, 507..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 67, 2304, 0, 0,..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100...","[1131, 89, 178, 108, 313, 93, 16, 4, 0, 13, 47..."


In [107]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,0.0,767400.0,彼岸领域
4,No Info Available,['No Info Available'],Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,No Info Available,"['Action', 'Indie', 'Casual', 'Sports']",http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,0.0,773570.0,No Info Available


In [108]:
genre_ranking = pd.DataFrame()
games = games.dropna()
items_playtime = items[['item_id', 'playtime_forever']].copy()
items_playtime = items_playtime.explode(['item_id', 'playtime_forever'])
items_playtime = items_playtime.dropna()
items_playtime['item_id'] = items_playtime['item_id'].astype(int)

In [109]:
items_playtime.head()

,item_id,playtime_forever
0,10,6
0,20,0
0,30,7
0,40,0
0,50,0


In [110]:
merged_data = pd.merge(games[['genres', 'id']], items_playtime, how='inner', left_on='id', right_on='item_id')

In [111]:
merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)

In [112]:
merged_data = merged_data.explode('genres')

In [113]:
merged_data = merged_data[['genres', 'playtime_forever']]
genre_rank = merged_data.groupby('genres')['playtime_forever'].sum().reset_index()
genre_rank['playtime_rank'] = genre_rank['playtime_forever'].rank(ascending=False).astype(int)
genre_rank = genre_rank.sort_values(by = 'playtime_rank', ascending=True)

In [114]:
genre_rank.head()

,genres,playtime_forever,playtime_rank
0,Action,3113529000,1
9,Indie,1494561494,2
13,RPG,1041010134,3
1,Adventure,909994325,4
15,Simulation,867645103,5


In [115]:
def genre(genre:str):
    row = genre_rank[genre_rank['genres'].str.contains(genre, case=False, na=False)]
    
    if row.empty:
        return "Género no encontrado"
    
    playtime_rank = row.iloc[0]['playtime_rank']

    result_dict = {
                    'genre': genre,
                    'playtime_rank': playtime_rank
    }
    
    return result_dict

In [116]:
genre('Simulation')

{'genre': 'Simulation', 'playtime_rank': 5}

In [117]:
genre_rank.to_csv('./API_csv/genre_genre_rank.csv')

## def userforgenre

In [118]:
items.head(1)

,user_id,items_count,steam_id,user_url,items,item_id,playtime_forever
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38...","[6, 0, 7, 0, 0, 0, 0, 0, 4733, 1853, 333, 75, ..."


In [119]:
items_user_playtime = items[['item_id', 'playtime_forever', 'user_id', 'user_url']].copy()
items_user_playtime = items_user_playtime.explode(['item_id', 'playtime_forever'])
items_user_playtime = items_user_playtime.dropna()
items_user_playtime['item_id'] = items_playtime['item_id'].astype(int)

In [120]:
items_user_playtime[items_user_playtime['user_id']=='js41637']

,item_id,playtime_forever,user_id,user_url
1,10,0,js41637,http://steamcommunity.com/id/js41637
1,80,0,js41637,http://steamcommunity.com/id/js41637
1,100,0,js41637,http://steamcommunity.com/id/js41637
1,300,220,js41637,http://steamcommunity.com/id/js41637
1,30,0,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...
1,473450,831,js41637,http://steamcommunity.com/id/js41637
1,485330,237,js41637,http://steamcommunity.com/id/js41637
1,485890,251,js41637,http://steamcommunity.com/id/js41637
1,491330,285,js41637,http://steamcommunity.com/id/js41637


In [121]:
id_genres = games[['id', 'genres']].copy()
id_genres['genres'] = id_genres['genres'].apply(ast.literal_eval)
id_genres = id_genres.explode('genres')
id_genres['id'] = id_genres['id'].astype(int)

In [122]:
id_genres.head()

,id,genres
0,761140,Action
0,761140,Casual
0,761140,Indie
0,761140,Simulation
0,761140,Strategy


In [123]:
merged_genres_playtime = pd.merge(id_genres, items_user_playtime, how='inner', left_on='id', right_on='item_id')

In [124]:
merged_genres_playtime.head()

,id,genres,item_id,playtime_forever,user_id,user_url
0,282010,Action,282010,5,UTNerd24,http://steamcommunity.com/id/UTNerd24
1,282010,Action,282010,0,I_DID_911_JUST_SAYING,http://steamcommunity.com/id/I_DID_911_JUST_SA...
2,282010,Action,282010,0,76561197962104795,http://steamcommunity.com/profiles/76561197962...
3,282010,Action,282010,0,r3ap3r78,http://steamcommunity.com/id/r3ap3r78
4,282010,Action,282010,13,saint556,http://steamcommunity.com/id/saint556


In [125]:
playtime_rank = merged_genres_playtime.groupby(['genres', 'user_id', 'user_url'])['playtime_forever'].sum().reset_index()

In [126]:
playtime_rank.sort_values(by=['genres', 'playtime_forever'], ascending=[True, False], inplace=True)
playtime_rank['rank'] = playtime_rank.groupby('genres').cumcount() + 1

In [127]:
playtime_rank.head()

,genres,user_id,user_url,playtime_forever,rank
50657,Action,Sp3ctre,http://steamcommunity.com/id/Sp3ctre,1699307,1
64497,Action,shinomegami,http://steamcommunity.com/id/shinomegami,1580189,2
49457,Action,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,1456212,3
51237,Action,Terminally-Chill,http://steamcommunity.com/id/Terminally-Chill,1065584,4
44133,Action,DownSyndromeKid,http://steamcommunity.com/id/DownSyndromeKid,1061193,5


In [128]:
def userforgenre(genre: str):
    genre_data = playtime_rank[playtime_rank['genres'] == genre].head(5)

    result_dict = genre_data.to_dict(orient='records')

    return result_dict

In [129]:
userforgenre('Action')

[{'genres': 'Action',
  'user_id': 'Sp3ctre',
  'user_url': 'http://steamcommunity.com/id/Sp3ctre',
  'playtime_forever': 1699307,
  'rank': 1},
 {'genres': 'Action',
  'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami',
  'playtime_forever': 1580189,
  'rank': 2},
 {'genres': 'Action',
  'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T',
  'playtime_forever': 1456212,
  'rank': 3},
 {'genres': 'Action',
  'user_id': 'Terminally-Chill',
  'user_url': 'http://steamcommunity.com/id/Terminally-Chill',
  'playtime_forever': 1065584,
  'rank': 4},
 {'genres': 'Action',
  'user_id': 'DownSyndromeKid',
  'user_url': 'http://steamcommunity.com/id/DownSyndromeKid',
  'playtime_forever': 1061193,
  'rank': 5}]

In [130]:
playtime_rank.to_csv('./API_csv/userforgenre_playtime_rank.csv')

## def developer

In [131]:
developer_year_price = games[['release_date', 'price', 'developer']].copy()
developer_year_price['release_date'] = developer_year_price['release_date'].str.extract(r'(\d{4})').fillna(0).astype(int)
non_numeric_rows = developer_year_price.loc[pd.to_numeric(developer_year_price['price'], errors='coerce').isna()]

In [132]:
grouped_developer_no_numeric = non_numeric_rows.groupby(['developer', 'release_date'])['price'].count().reset_index()
grouped_developer_no_numeric = grouped_developer_no_numeric.sort_values(by = ['developer', 'release_date'], ascending=True)

grouped_developer = developer_year_price.groupby(['developer', 'release_date'])['price'].size().reset_index()
grouped_developer = grouped_developer.sort_values(by = ['developer', 'release_date'], ascending=True)

grouped_developer_no_numeric = grouped_developer_no_numeric.rename(columns={'price': 'count_free'})
grouped_developer = grouped_developer.rename(columns={'price': 'total_count'})

In [133]:
merged_developer = pd.merge(grouped_developer, grouped_developer_no_numeric, how='left', on=['developer', 'release_date'])
merged_developer = merged_developer.rename(columns={'release_date': 'year'})
merged_developer['pct_free'] = merged_developer['count_free'] / merged_developer['total_count']
merged_developer['pct_free'].fillna(0, inplace=True)

In [134]:
def developer(developer: str):
    result_df = merged_developer[merged_developer['developer'] == developer].reset_index()

    result_dict = result_df[['year', 'pct_free']].to_dict(orient='records')
    return result_dict

In [135]:
developer('Activision')

[{'year': 1993, 'pct_free': 0.0},
 {'year': 1996, 'pct_free': 0.0},
 {'year': 1997, 'pct_free': 0.0},
 {'year': 2000, 'pct_free': 0.0}]

In [136]:
merged_developer.to_csv('./API_csv/developer_merged_developer.csv')

## def sentiment_analysis

In [137]:
reviews_sentiment = reviews[['user_id','item_id', 'sentiment_analysis']].copy()
reviews_sentiment = reviews_sentiment.explode(['item_id','sentiment_analysis'])
reviews_sentiment['item_id'].fillna(0, inplace=True)
reviews_sentiment['item_id'] = reviews_sentiment['item_id'].astype(int)

In [138]:
sentiment_analysis_developer = pd.merge(reviews_sentiment, games[['id','developer', 'release_date']], how='inner', left_on='item_id', right_on='id')
sentiment_analysis_developer['release_date'] = sentiment_analysis_developer['release_date'].str.extract(r'(\d{4})').fillna(0).astype(int)
sentiment_analysis_developer = sentiment_analysis_developer.drop(columns=['user_id', 'item_id', 'id'])
sentiment_analysis_developer = sentiment_analysis_developer.sort_values(by = ['developer', 'release_date'], ascending=[True, True])

In [139]:
sentiment_analysis_developer.head()

,sentiment_analysis,developer,release_date
52082,0,07th Expansion,2015
52083,1,07th Expansion,2015
52636,1,07th Expansion,2016
53443,1,"10th Art Studio,Adventure Productions",2014
53444,1,"10th Art Studio,Adventure Productions",2014


In [140]:
def sentiment_analysis(year: int):
    
    year_df = sentiment_analysis_developer[sentiment_analysis_developer['release_date'] == year]
    
    counts = year_df['sentiment_analysis'].value_counts().to_dict()
    
    for sentiment in [0, 1, 2]:
        if sentiment not in counts:
            counts[sentiment] = 0
    
    sentiment_dict = {
        'Negative': counts['0'],
        'Neutral': counts['1'],
        'Positive': counts['2']
    }
    
    return sentiment_dict

In [141]:
sentiment_analysis(2011)

{'Negative': 480, 'Neutral': 1408, 'Positive': 1566}

In [142]:
sentiment_analysis_developer.to_csv('./API_csv/sentiment_analysis_sentiment_analysis_developer.csv')